# Библиотеки

In [1]:
import neptune
import pandas as pd

from cv import get_indices
from load_data import load
from model_selection import training
import random
from NN import simple_torchpl
from pl_framework import nn_training
from pytorch_forecasting.metrics import MAPE
import torch

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="3334104b-e4da-40ca-a353-d78790164c0b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321

# Загрузка данных

In [5]:
train_features, train_targets, _ = load(data_path)

# Валидация

In [6]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [7]:
#train_targets = train_targets.drop('timestamp', axis = 'columns')
#train_features = train_features.drop('timestamp', axis = 'columns')

# Подбор модели

In [ ]:
my_training = training(name = 'NN', nn_model = simple_torchpl, training_nn = nn_training,
                       description = 'Ffill. 1 train set for all folds. 1 model for all targets. LGBM. \
                       Loss = MAPE. No normalisation. Only first 2 fold and test set(last fold).Dropped data before 2020-02-15 00:00:00',
                       upload_source_files = ['cv.py', 
                                              'load_data.py', 
                                              'model_selection.py',
                                              'NN.py',
                                              'pl_framework.py'])
my_training.set_up_studying(random_state = random_state)

model = 'lgbm'

def params_func(trial, X):
    return(
{        'objective': 'regression',
        'metric': 'mape',
        'n_jobs': -1,
        'n_estimators': 10,
        'random_state': random_state,
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'n_in': trial.suggest_int('n_in', 1, 150),
        'n_back': trial.suggest_int('n_back', 1, 150)}
    )

n_trials = 100
my_training.train(X = train_features, 
                  y = train_targets, 
                  cv = cv, 
                  model=model, 
                  params_func = params_func, 
                  n_trials = n_trials)

https://ui.neptune.ai/iliaavilov/SIBUR/e/SIB-232


[I 2020-11-29 02:58:20,077] A new study created in memory with name: no-name-8572bc66-4610-40a1-afa8-261b8db1a79d


current params: {'objective': 'regression', 'metric': 'mape', 'n_jobs': -1, 'n_estimators': 10, 'random_state': 54321, 'num_leaves': 83, 'learning_rate': 0.12858766698944454, 'min_child_samples': 98, 'feature_fraction': 0.8750823012386388, 'bagging_fraction': 0.6578708780010873, 'n_in': 11, 'n_back': 128}
X shape before dropped nans (3647, 11)
X shape after dropped nans (3519, 11)
cv shape before dropped nans 719
cv shape after dropped nans 580
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30600
[LightGBM] [Info] Number of data points in the train set: 580, number of used features: 120
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30600
[LightGBM] [Info] Number of data points in the train set: 5

[I 2020-11-29 02:58:49,884] Trial 0 finished with value: 0.09263365896757793 and parameters: {'num_leaves': 83, 'learning_rate': 0.12858766698944454, 'min_child_samples': 98, 'feature_fraction': 0.8750823012386388, 'bagging_fraction': 0.6578708780010873, 'n_in': 11, 'n_back': 128}. Best is trial 0 with value: 0.09263365896757793.


current params: {'objective': 'regression', 'metric': 'mape', 'n_jobs': -1, 'n_estimators': 10, 'random_state': 54321, 'num_leaves': 100, 'learning_rate': 0.1776880853574783, 'min_child_samples': 193, 'feature_fraction': 0.7230560757101125, 'bagging_fraction': 0.42488166078719586, 'n_in': 62, 'n_back': 14}
X shape before dropped nans (3647, 11)
X shape after dropped nans (3633, 11)
cv shape before dropped nans 719
cv shape after dropped nans 643
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 160647
[LightGBM] [Info] Number of data points in the train set: 643, number of used features: 630
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 160647
[LightGBM] [Info] Number of data points in the train set

...

cv shape after dropped nans 638
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28050
[LightGBM] [Info] Number of data points in the train set: 638, number of used features: 110
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28050
[LightGBM] [Info] Number of data points in the train set: 638, number of used features: 110
[LightGBM] [Info] Start training from score 4.312528
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Start training from score 4.312528
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[I 2020-11-29 03:03:15,690] Trial 6 finished with value: 0.09200670611231761 and parameters: {'num_leaves': 95, 'learning_rate': 0.2533985023319476, 'min_child_samples': 68, 'feature_fraction': 0.6797975595123207, 'bagging_fraction': 0.4624139070665466, 'n_in': 79, 'n_back': 90}. Best is trial 1 with value: 0.09060085541330458.


Auto-choosing col-wise multi-threading, the overhead of testing was 0.023220 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 147200
[LightGBM] [Info] Number of data points in the train set: 550, number of used features: 800
[LightGBM] [Info] Start training from score 4.312250
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
current params: {'objective': 'regression', 'metric': 'mape', 'n_jobs': -1, 'n_estimators': 10, 'random_state': 54321, 'num_leaves': 256, 'learning_rate': 0.10785544767261263, 'min_child_samples': 74, 'feature_fraction': 0.40503045729245485, 'bagging_fraction': 0.914330770409679, 'n_in': 21, 'n_back': 17}
X shape before dropped nans (3647, 11)
X shape after dropped nans (3630, 11)
cv shape before dropped nans 719
cv shape after dropped nans 681
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of

[I 2020-11-29 03:03:50,410] Trial 7 finished with value: 0.09202113765505691 and parameters: {'num_leaves': 256, 'learning_rate': 0.10785544767261263, 'min_child_samples': 74, 'feature_fraction': 0.40503045729245485, 'bagging_fraction': 0.914330770409679, 'n_in': 21, 'n_back': 17}. Best is trial 1 with value: 0.09060085541330458.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50160
[LightGBM] [Info] Number of data points in the train set: 681, number of used features: 220
[LightGBM] [Info] Start training from score 4.307991
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
current params: {'objective': 'regression', 'metric': 'mape', 'n_jobs': -1, 'n_estimators': 10, 'random_state': 54321, 'num_leaves': 119, 'learning_rate': 0.26749465811232825, 'min_child_samples': 56, 'feature_fraction': 0.8849333789478713, 'bagging_fraction': 0.46171120010460803, 'n_in': 134, 'n_back': 40}
X shape before dropped nans (3647, 11)
X shape after dropped nans (3607, 11)
cv shape before dropped nans 719


In [ ]:
#neptune.log_metric('leaderboard_mape', 3.2443)
#neptune.project.get_experiments('SIB-205')[0].stop()
#neptune.stop()